In [1]:
import os
import os.path as osp
import networkx as nx
import json
import matplotlib.pyplot as plt
import hashlib
import shutil
import subprocess
from tqdm import tqdm

from utils.new_map_utils import get_game_info
from utils.clean_utils import compute_hash_for_path,compute_hash_for_pair
from utils.utils import find_json,read_json

In [2]:
# notebook input
map_dir='/share/data/mei-work/kangrui/github/mango/data/'
result_dir='/share/data/mei-work/kangrui/github/mango/kangrui/data/gpt-games-results-clean-new'

In [3]:
game_names=set()
for game_name in os.listdir(map_dir):
    game_names.add(game_name)
json_files=find_json(result_dir)

for json_file in tqdm(json_files):
    
    try:
        data=read_json(json_file)
    except Exception as e:
        print(json_file,e)
        os.sys.exit()
    
#     if data["task"]=="pathgen":
#         continue
        
        
    game_name=None
    for name in json_file.split('/'):
        if name in game_names:
            game_name=name
            break
    if game_name==None:
        continue
    G,actions,locations,all2all,all_pairs,walkthrough=get_game_info(map_dir,game_name)
      
    if data["task"]=="pathgen":
        
        hashcode=compute_hash_for_pair({"src_node":data["src_node"],"dst_node":data["dst_node"]})
        try:
            assert hashcode in set([pair["id"] for pair in all_pairs])
        except Exception as e:
            print(game_name,json_file,e)
            os.sys.exit()
        data["task_id"]=hashcode

    
    if data["task"]=="stepnav":
        hashcode=compute_hash_for_path(data["path_gt"])
        try:
            assert hashcode in set([path["id"] for path in all2all])
        except Exception as e:
            print(game_name,json_file,e)
            os.sys.exit()
        data["task_id"]=hashcode

    try:
        with open(json_file, 'w') as f:
            json.dump(data,f,indent=4)
    except Exception as e:
        print(e)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 41920/41920 [12:36<00:00, 55.42it/s]
